In [ ]:
// Nuget Packages
#r "nuget: MelonChart.NET, 1.*"

#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"


In [2]:
// Add using statements
using System.ComponentModel;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [3]:
// Azure OpenAI configurations
var endpoint = "<AZURE_OPENAI_ENDPOINT>";
var apiKey = "<AZURE_OPENAI_API_KEY>";
var deploymentName = "<AZURE_OPENAI_DEPLOYMENT_NAME>";

In [4]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [5]:
// Import prompts
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts");

In [6]:
// Import codes
#!import Plugins/AddMemory/AddMelonChartPlugin.cs

In [7]:
// Import plugins
kernel.ImportPluginFromType<AddMelonChartPlugin>();

In [8]:
// Build Semantic Memory
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
                 .WithAzureOpenAITextEmbeddingGeneration(
                     deploymentName: "model-textembeddingada002-2",
                     endpoint: endpoint,
                     apiKey: apiKey)
                 .WithMemoryStore(new VolatileMemoryStore())
                 .Build();

In [9]:
// Add HttpClient instance.
var http = new HttpClient();

In [10]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [ ]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("무엇이 궁금한가요?");

Console.WriteLine($"User: {question}");

In [ ]:
// Invoke the prompt - GetIntent
var intent = await kernel.InvokeAsync<string>(
                        function: prompts["GetIntent"],
                        arguments: new KernelArguments()
                        {
                            { "input", question }
                        });

Console.WriteLine(intent);

In [ ]:
// Invoke the prompt - RefineQuestion
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineQuestion"],
                        arguments: new KernelArguments()
                        {
                            { "input", question },
                            { "intent", intent }
                        });

Console.WriteLine(refined);

In [ ]:
// Invoke the plugin - Add Melon Chart data
await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.AddChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

In [ ]:
// Invoke the plugin - Find songs
var results = await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.FindSongs), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "question", refined },
        { "jso", jso }, 
    }
);

var data = results.GetValue<List<ChartItem>>().Select(p => JsonSerializer.Serialize(p, jso)).Aggregate((x, y) => $"{x}\n{y}");

Console.WriteLine(data);

In [ ]:
// Invoke the prompt - RefineResult
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineResult"],
                        arguments: new KernelArguments()
                        {
                            { "input", data },
                            { "intent", intent }
                        });

Console.WriteLine(refined);